### Importing necessary modules

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy
import sep
from astropy.io import fits
from matplotlib import rcParams

rcParams['figure.figsize']=[10.,8.]

In [ ]:
#reading the image into a 2-d np array
data = astropy.io.fits.getdata("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data = data.byteswap().newbyteorder()

In [ ]:
check_info_to_convertAB = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
hdu_list = fits.open(check_info_to_convertAB)
hdu_list.info()
image_data = hdu_list[0].data
image_header = hdu_list[0].header
print(image_header)

In [ ]:
#show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

In [ ]:
#measure a spatially varying background on the image

bkg = sep.Background(data)

In [ ]:
#get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate the background as 2-d array, same size as original image
#bkg_image = bkg.back()
bkg_image = np.array(bkg)

In [ ]:
#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#evaluate the background noise as 2-d array, ssame size as original image
bkg_rms = bkg.rms()

In [ ]:
#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#background subtraction
data_sub = data - bkg

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

## This is how many sources!!! wowwww a lot

In [ ]:
#how many objects that are detected by the abve by taking the length of the array
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

#plot the background subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('yellow')
    ax.add_artist(e)

# Spongebob Galaxy Pants

In [ ]:
#available fields
objects.dtype.names

## Aperture photometry

In [ ]:
#set 1-d arrays with 1 element each
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:F} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
print(flux)

## Converting flux to AB magnitude units
So we found the formula below that is commented out first, but then the professor told us to use the other website, which we had to do some digging to find the value to add on, and Michelle added a .jpg of that chart. We also had to use a forloop to make the flux values all positive. The correction is from another chart, the corrections for all the different filters were about the same.

In [ ]:

# Convert flux to AB magnitude units
#flux = -2.5*np.log10(flux/3631)
#print(flux)

In [ ]:
correction_inf = 0.877
flux_inf = flux/correction_inf

for i in range(len(flux)):
    if flux_inf[i]>0:
        n = -2.5*np.log10(flux_inf[i]) + 26.2687
        flux[i] = n
        #print(n)
    elif flux_inf[i]<0:
        n = -2.5 * np.log10(-flux_inf[i]) + 26.2687
        flux[i] = n
        #print(n)

In [ ]:
#m_ab = -2.5 * np.log10(flux) + 26.2687
#this is commented out because of the negative values, corrected by the above for loop

In [ ]:
#n_bins=10
#plot a histogram by flattening the data from the 2-d array to a 1-d numpy array
histogram = plt.hist(flux, bins='auto')

## RGB false image
I just used the make lupton module

In [ ]:
from astropy.visualization import make_lupton_rgb

In [ ]:

image_f105w = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
image_f125w = "hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
image_f160w = "hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"
f105w_data = fits.getdata(image_f105w)
f125w_data = fits.getdata(image_f125w)
f160w_data = fits.getdata(image_f160w)

In [ ]:
# Create the png file and plot the rgb image

rgb_image = make_lupton_rgb(f160w_data,f125w_data,f105w_data,
                           filename="hduf_rgb_false_image.png")
plt.imshow(rgb_image, interpolation='nearest', vmin=m-s,vmax=m+s, origin='lower')
plt.colorbar()

In [ ]:
#Adjusting image so we can actually see it
# Q adjusts smoothness and stretch zooms in

rgb = make_lupton_rgb(f160w_data,f125w_data,f105w_data, Q=0.2, stretch=0.005,
                     filename='hduf_rbg_false_image(adjusted).png')

# Plot the new image
plt.imshow(rgb, origin='lower')
plt.colorbar()

In [ ]:
#plotting just a portion of the above image

rgb = make_lupton_rgb(f160w_data[1000:2500,1000:2500],
                     f125w_data[1000:2500,1000:2500],
                     f105w_data[1000:2500, 1000:2500],
                     Q=0.02, stretch=0.005,
                     filename='hduf_rbg_false_image(zoomed).png')
plt.imshow(rgb, origin='lower')
plt.colorbar()